In [162]:
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings('ignore')
con = sqlite3.connect("database.sqlite")

def show_query(query):
    """
       Deze functie print de output van je SQL query uit.
       Parameters:
                  query = Je SQL query.
                          Zorg ervoor dat deze parameter een string is.
    """
    query_output = pd.read_sql_query(query, con)
    return pd.DataFrame(query_output)


In [163]:
# Load the data into a DataFrame
df_country = show_query("SELECT * FROM Country")
df_league = show_query("SELECT * FROM League")
df_match = show_query("SELECT * FROM Match")
df_player = show_query("SELECT * FROM Player")
df_player_attributes = show_query("SELECT * FROM Player_Attributes")
df_team = show_query("SELECT * FROM Team")
df_team_attributes = show_query("SELECT * FROM Team_Attributes")

Maak een functie waarmee je per seizoen voor een gekozen club o.a. het volgende kunt bepalen:

- Aantal wedstrijden in dat seizoen.
- Aantal punten behaald in dat seizoen.
- Gemiddeld aantal punten per wedstrijd.
- Winst-, verlies- en gelijk percentage in uitwedstrijden en thuiswedstrijden.

In [173]:
def matchinfo(teamid, season):
    home_match_results = []
    home_team_points = []
    homedf = df_match[(df_match['home_team_api_id'] == teamid) & (df_match['season'] == season)]
    for index,row in homedf.iterrows():
        if row['home_team_goal'] > row['away_team_goal']:
            home_match_results.append('win')
            home_team_points.append(3)
        elif row['home_team_goal'] == row['away_team_goal']:
            home_match_results.append('draw')
            home_team_points.append(1)
        else:
            home_match_results.append('loss')
            home_team_points.append(0)
   
   
    homedf['match_results'] = home_match_results
    homedf['team_points'] = home_team_points
   
    away_match_results = []
    away_team_points = []
    awaydf = df_match[(df_match['away_team_api_id'] == teamid) & (df_match['season'] == season)]
    for index,row in awaydf.iterrows():
        if row['away_team_goal'] > row['home_team_goal']:
            away_match_results.append('win')
            away_team_points.append(3)
        elif row['away_team_goal'] == row['home_team_goal']:
            away_match_results.append('draw')
            away_team_points.append(1)
        else:
            away_match_results.append('loss')
            away_team_points.append(0)
   
    awaydf['match_results'] = away_match_results
    awaydf['team_points'] = away_team_points
   
    homepoints = homedf['team_points'].sum()
    awaypoints = awaydf['team_points'].sum()
    
    totalpoints = homepoints + awaypoints
    totalmatch = len(homedf) + len(awaydf)
    averagepoints = totalpoints / totalmatch
    
    totalmatch_home = len(homedf) 
    if totalmatch_home > 0:
        perc_win_home = len(homedf[homedf['team_points'] == 3]) / totalmatch_home * 100
        perc_draw_home = len(homedf[homedf['team_points'] == 1]) / totalmatch_home * 100
        perc_loss_home = len(homedf[homedf['team_points'] == 0]) / totalmatch_home * 100
    else: 
        perc_win_home = 0
        perc_draw_home = 0
        perc_loss_home = 0
    
    totalmatch_away = len(awaydf)
    if totalmatch_away > 0:
        perc_win_away = len(awaydf[awaydf['team_points'] == 3]) / totalmatch_away * 100
        perc_draw_away = len(awaydf[awaydf['team_points'] == 1]) / totalmatch_away * 100
        perc_loss_away = len(awaydf[awaydf['team_points'] == 0]) / totalmatch_away * 100
    else: 
        perc_win_away = 0
        perc_draw_away = 0
        perc_loss_away = 0
    
    df = pd.DataFrame()
    df['season'] = [season]
    df['team'] = teamid
    df['total_games'] = totalmatch
    df['total_points'] = totalpoints
    df['average_points'] = averagepoints
    df['perc_win_home'] = perc_win_home
    df['perc_draw_home'] = perc_draw_home
    df['perc_loss_home'] = perc_loss_home
    df['perc_win_away'] = perc_win_away
    df['perc_draw_away'] = perc_draw_away
    df['perc_loss_away'] = perc_loss_away
    
    return df

In [174]:
matchinfo(10228, '2008/2009')

,season,team,total_games,total_points,average_points,perc_win_home,perc_draw_home,perc_loss_home,perc_win_away,perc_draw_away,perc_loss_away
0,2008/2009,10228,34,60,1.764706,64.705882,29.411765,5.882353,35.294118,23.529412,41.176471


In [179]:
team_list = list(df_match['home_team_api_id'].unique())

df_season = pd.DataFrame()
for team in team_list:
    df_season.concat(team, '2008/2009')
    


AttributeError: 'DataFrame' object has no attribute 'concat'